# Baseline-решение

По мотивам ноутбука https://www.kaggle.com/code/hardtype/parsing-news-from-rbc-lenta-ru

## 1. Парсим новости с сайта Lenta.ru

In [1]:
import time
# Установка библиотек
!pip install bs4
!pip install openpyxl

In [56]:
# Импорт библиотек
import requests as rq
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
# from IPython import display
import time

In [68]:
rubrics = {
    'society_russia' : 1,
    'economics': 2,
    'forces_army': 37,
    'ussr': 3,
    'sports': 8,
    'selfcare': 87,
    'tourism': 48,
    'science': 8
}

rubrics_w_building = rubrics.copy()
rubrics_w_building.update({'building' : 999})
rubrics_w_building

{'society_russia': 1,
 'economics': 2,
 'forces_army': 37,
 'ussr': 3,
 'sports': 8,
 'selfcare': 87,
 'tourism': 48,
 'science': 8,
 'building': 999}

In [60]:
class lentaRu_parser:
    def __init__(self):
        pass

    def _get_url(self, param_dict: dict) -> str:
        """
        Возвращает URL для запроса json таблицы со статьями

        url = 'https://lenta.ru/search/v2/process?'\
        + 'from=0&'\                       # Смещение
        + 'size=1000&'\                    # Кол-во статей
        + 'sort=2&'\                       # Сортировка по дате (2), по релевантности (1)
        + 'title_only=0&'\                 # Точная фраза в заголовке
        + 'domain=1&'\                     # ??
        + 'modified%2Cformat=yyyy-MM-dd&'\ # Формат даты
        + 'type=1&'\                       # Материалы. Все материалы (0). Новость (1)
        + 'bloc=4&'\                       # Рубрика. Экономика (4). Все рубрики (0)
        + 'modified%2Cfrom=2020-01-01&'\
        + 'modified%2Cto=2020-11-01&'\
        + 'query='                         # Поисковой запрос
        """
        hasType = int(param_dict['type']) != 0
        hasBloc = int(param_dict['bloc']) != 0

        url = 'https://lenta.ru/search/v2/process?'\
        + 'from={}&'.format(param_dict['from'])\
        + 'size={}&'.format(param_dict['size'])\
        + 'sort={}&'.format(param_dict['sort'])\
        + 'title_only={}&'.format(param_dict['title_only'])\
        + 'domain={}&'.format(param_dict['domain'])\
        + 'modified%2Cformat=yyyy-MM-dd&'\
        + 'type={}&'.format(param_dict['type']) * hasType\
        + 'bloc={}&'.format(param_dict['bloc']) * hasBloc\
        + 'modified%2Cfrom={}&'.format(param_dict['dateFrom'])\
        + 'modified%2Cto={}&'.format(param_dict['dateTo'])\
        + 'query={}'.format(param_dict['query'])
        print(url)
        return url


    def _get_search_table(self, param_dict: dict) -> pd.DataFrame:
        """
        Возвращает pd.DataFrame со списком статей
        """
        url = self._get_url(param_dict)
        r = rq.get(url)
        search_table = pd.DataFrame(r.json()['matches'])

        total = r.json()['total']
        size = param_dict.get('size', 10)
        current_from = param_dict.get('from', 0)

        if total >= 1000:
            print('parse more!!!')
            additional_requests = []
            for i in range(1, 7):  # 5 дополнительных запросов
                new_from = current_from + size * i
                new_params = param_dict.copy()
                new_params['from'] = new_from
                url = self._get_url(new_params)
                additional_requests.append(url)

            for url in additional_requests:
                    try:
                        r = rq.get(url)
                        matches = r.json()['matches']
                        if matches:
                            df = pd.DataFrame(matches)
                            search_table = pd.concat([search_table, df], ignore_index=True)
                            time.sleep(1)
                    except Exception as e:
                        print(f"Ошибка при обработке URL {url}: {e}")
        return search_table


    def get_articles(self,
                     param_dict,
                     time_step = 37,
                     save_every = 5,
                     save_excel = True) -> pd.DataFrame:
        """
        Функция для скачивания статей интервалами через каждые time_step дней
        Делает сохранение таблицы через каждые save_every * time_step дней

        param_dict: dict
        ### Параметры запроса
        ###### project - раздел поиска, например, rbcnews
        ###### category - категория поиска, например, TopRbcRu_economics
        ###### dateFrom - с даты
        ###### dateTo - по дату
        ###### offset - смещение поисковой выдачи
        ###### limit - лимит статей, максимум 100
        ###### query - поисковой запрос (ключевое слово), например, РБК

        """
        param_copy = param_dict.copy()
        time_step = timedelta(days=time_step)
        dateFrom = datetime.strptime(param_copy['dateFrom'], '%Y-%m-%d')
        dateTo = datetime.strptime(param_copy['dateTo'], '%Y-%m-%d')
        if dateFrom > dateTo:
            raise ValueError('dateFrom should be less than dateTo')

        out = pd.DataFrame()
        save_counter = 0

        while dateFrom <= dateTo:
            param_copy['dateTo'] = (dateFrom + time_step).strftime('%Y-%m-%d')
            if dateFrom + time_step > dateTo:
                param_copy['dateTo'] = dateTo.strftime('%Y-%m-%d')
            print('Parsing articles from '\
                  + param_copy['dateFrom'] +  ' to ' + param_copy['dateTo'])
            out = pd.concat([out, self._get_search_table(param_copy)], ignore_index=True)
            dateFrom += time_step + timedelta(days=1)
            param_copy['dateFrom'] = dateFrom.strftime('%Y-%m-%d')
            save_counter += 1
            if save_counter == save_every:
                # display.clear_output(wait=True)
                out.to_excel("/tmp/checkpoint_table.xlsx")
                print('Checkpoint saved!')
                save_counter = 0

        if save_excel:
            out.to_excel("lenta_{}_{}.xlsx".format(
                param_dict['dateFrom'],
                param_dict['dateTo']))
        print('Finish')

        return out

In [61]:
# Задаем тут параметры
query = ''
offset = 0
size = 100
sort = "3"
title_only = "0"
domain = "1"
material = "0"
bloc = "0"
dateFrom = '2023-01-01'
dateTo = "2025-01-01"

param_dict = {'query'     : query,
              'from'      : str(offset),
              'size'      : str(size),
              'dateFrom'  : dateFrom,
              'dateTo'    : dateTo,
              'sort'      : sort,
              'title_only': title_only,
              'type'      : material,
              'bloc'      : bloc,
              'domain'    : domain}

print("param_dict:", param_dict)

param_dict: {'query': '', 'from': '0', 'size': '100', 'dateFrom': '2023-01-01', 'dateTo': '2025-01-01', 'sort': '3', 'title_only': '0', 'type': '0', 'bloc': '0', 'domain': '1'}


In [62]:
# Тоже будем собирать итеративно, правда можно ставить time_step побольше, т.к.
# больше лимит на запрос статей. И Работает быстрее :)

parser = lentaRu_parser()

for rubric in rubrics.items():
    param_dict['bloc'] = rubric[1]
    print(param_dict.get('bloc'))
    tbl = parser.get_articles(param_dict=param_dict,
                             time_step = 37,
                             save_every = 5,
                             save_excel = True)
    print(len(tbl.index), rubric[0])

    tbl.to_csv(f"Lenta_r_{str(rubric[0])}.csv", index=False)

1
Parsing articles from 2023-01-01 to 2023-02-07
https://lenta.ru/search/v2/process?from=0&size=100&sort=3&title_only=0&domain=1&modified%2Cformat=yyyy-MM-dd&bloc=1&modified%2Cfrom=2023-01-01&modified%2Cto=2023-02-07&query=
parse more!!!
https://lenta.ru/search/v2/process?from=0100&size=100&sort=3&title_only=0&domain=1&modified%2Cformat=yyyy-MM-dd&bloc=1&modified%2Cfrom=2023-01-01&modified%2Cto=2023-02-07&query=
https://lenta.ru/search/v2/process?from=0100100&size=100&sort=3&title_only=0&domain=1&modified%2Cformat=yyyy-MM-dd&bloc=1&modified%2Cfrom=2023-01-01&modified%2Cto=2023-02-07&query=
https://lenta.ru/search/v2/process?from=0100100100&size=100&sort=3&title_only=0&domain=1&modified%2Cformat=yyyy-MM-dd&bloc=1&modified%2Cfrom=2023-01-01&modified%2Cto=2023-02-07&query=
https://lenta.ru/search/v2/process?from=0100100100100&size=100&sort=3&title_only=0&domain=1&modified%2Cformat=yyyy-MM-dd&bloc=1&modified%2Cfrom=2023-01-01&modified%2Cto=2023-02-07&query=
https://lenta.ru/search/v2/proce

In [66]:
import pandas as pd
import glob
import os

def read_csv(file_path):
    """
    Читает CSV файл и возвращает DataFrame.
    """
    try:
        df = pd.read_csv(file_path)
        print(f"Успешно прочитан: {file_path}")
        return df
    except Exception as e:
        print(f"Ошибка при чтении {file_path}: {e}")
        return None

path = './'

# Создаем шаблон поиска файлов, начинающихся с 'Lenta_' и имеющих расширение .csv
file_pattern = os.path.join(path, 'Lenta_*.csv')
csv_files = glob.glob(file_pattern)

print(f"Найдено {len(csv_files)} файлов для объединения.")

# Чтение файлов один за другим
dataframes = []
for file in csv_files:
    df = read_csv(file)
    if df is not None:
        dataframes.append(df)

# Проверяем, что DataFrame не пустой
if dataframes:
    # Объединяем все DataFrame в один
    merged_df = pd.concat(dataframes, ignore_index=True)
    print("Все файлы успешно объединены.")
    print(f"Общий размер объединенного DataFrame: {merged_df.shape}")
else:
    print("Не удалось прочитать ни один файл.")

# (Опционально) Просмотр первых строк объединенного DataFrame
merged_df.head()


Найдено 8 файлов для объединения.
Успешно прочитан: ./Lenta_r_tourism.csv
Успешно прочитан: ./Lenta_r_forces_army.csv
Успешно прочитан: ./Lenta_r_selfcare.csv
Успешно прочитан: ./Lenta_r_society_russia.csv
Успешно прочитан: ./Lenta_r_science.csv
Успешно прочитан: ./Lenta_r_ussr.csv
Успешно прочитан: ./Lenta_r_sports.csv
Успешно прочитан: ./Lenta_r_economics.csv
Все файлы успешно объединены.
Общий размер объединенного DataFrame: (26400, 16)


,docid,url,title,modified,lastmodtime,type,domain,status,part,bloc,tags,image_url,pubdate,text,rightcol,snippet
0,1363840,https://lenta.ru/news/2023/01/01/poezd/,В России поставят точку в споре между пассажир...,1672554120,1673192348,1,1,0,0,48,[159],https://icdn.lenta.ru/images/2023/01/01/09/202...,1672554120,Фото: Александр Гальперин / РИА Новости Варвар...,В России поставят точку в споре между пассажир...,Фото: Александр Гальперин / РИА Новости ... на...
1,1363950,https://lenta.ru/news/2023/01/01/perelity/,Названы популярные у россиян страны и города д...,1672589820,1673192339,1,1,0,0,48,"[158, 159]",https://icdn.lenta.ru/images/2023/01/01/19/202...,1672589820,Фото: Evgenii Bugubaev / Anadolu Agency via G...,Названы популярные у россиян страны и города д...,Фото: Evgenii Bugubaev / Anadolu Agency ... н...
2,1364096,https://lenta.ru/news/2023/01/02/airastana_trash/,Пассажиры самолета попали в зону сильной турбу...,1672658940,1672660748,1,1,0,0,48,[4],https://icdn.lenta.ru/images/2023/01/02/14/202...,1672658940,Архивное фото Фото: Александр Гальперин / РИА ...,Пассажиры самолета попали в зону сильной турбу...,Архивное фото Фото: Александр Гальперин /... и...
3,1364122,https://lenta.ru/news/2023/01/02/turkey_whyyyy...,В Турции начал действовать налог на проживание...,1672665180,1673192408,1,1,0,0,48,[158],https://icdn.lenta.ru/images/2023/01/02/16/202...,1672665180,Фото: Artur Voznenko / Unsplash Варвара Кошечк...,В Турции начал действовать налог на проживание...,Фото: Artur Voznenko / Unsplash Варвара ... вз...
4,1364168,https://lenta.ru/news/2023/01/02/nalog/,В Риге ввели отложенный из-за пандемии налог н...,1672680250,1672680250,1,1,0,0,48,[158],https://icdn.lenta.ru/images/2023/01/02/20/202...,1672680250,Фото: Ints Kalnins / Reuters Никита Абрамов Вл...,В Риге ввели отложенный из-за пандемии налог н...,Фото: Ints Kalnins / Reuters Никита ... был от...


In [70]:
!pip install swifter -q


 Добавляем класс строительство при присутствии слова в заголовке статьи

In [91]:
import re
import string
import pandas as pd

def preprocess_text(text):
    """
    Предварительная обработка текста:
    - Если текст начинается со слова 'фото', удалить все слова до первого слеша.
    - Приведение текста к нижнему регистру.
    - Удаление знаков препинания.
    - Удаление лишних пробелов.

    :param text: Исходный текст.
    :return: Обработанный текст.
    """
    text = text.strip()

    # Проверка, начинается ли текст со слова 'фото' (без учета регистра)
    if text.lower().startswith('фото'):
        # Поиск первого слеша
        slash_index = text.find('/')
        if slash_index != -1:
            # Удаление всего до первого слеша (включая слеш)
            text = text[slash_index + 1:].strip()

    # Приведение к нижнему регистру
    text = text.lower()

    # Удаление знаков препинания
    punctuation_pattern = f"[{re.escape(string.punctuation)}]"
    text = re.sub(punctuation_pattern, "", text)

    # Удаление лишних пробелов
    text = re.sub(r'\s+', ' ', text).strip()

    return text

def preprocess_dataframe(df, columns, primary_column='title', secondary_column='text'):
    """
    Обрабатывает указанные столбцы DataFrame:
    1. Если 'text' пустой, заполняет его значением из 'title'.
    2. Если оба 'title' и 'text' пустые, удаляет запись.
    3. Предварительная обработка 'text':
       - Если начинается со слова 'фото', удалить все до первого слеша.
    4. Приведение текста к нижнему регистру.
    5. Удаление знаков препинания.
    6. Удаление лишних пробелов.

    :param df: pandas DataFrame
    :param columns: Список столбцов для обработки (обычно ['title', 'text'])
    :param primary_column: Основной столбец для заполнения (по умолчанию 'title')
    :param secondary_column: Вторичный столбец для заполнения (по умолчанию 'text')
    :return: Обработанный DataFrame
    """
    # Создаем копию DataFrame для избежания изменения исходных данных
    df = df.copy()

    # Заполняем значения NaN пустыми строками
    for column in columns:
        if column in df.columns:
            df[column] = df[column].fillna('')
        else:
            print(f"Столбец '{column}' не найден в DataFrame.")

    # Заполняем пустые значения 'text' значениями из 'title'
    df['text'] = df.apply(
        lambda row: row[primary_column] if not row[secondary_column].strip() else row[secondary_column],
        axis=1
    )

    # Удаляем записи, где оба поля пустые после заполнения
    df = df[~((df[primary_column].str.strip() == '') & (df['text'].str.strip() == ''))]

    # Применяем предварительную обработку к столбцу 'text'
    df['text'] = df['text'].apply(preprocess_text)

    # Обработка остальных текстовых данных
    for column in columns:
        if column in df.columns:
            if column != 'text':
                # Приведение текста к нижнему регистру
                df[column] = df[column].str.lower()

                # Удаление знаков препинания
                punctuation_pattern = f"[{re.escape(string.punctuation)}]"
                df[column] = df[column].str.replace(punctuation_pattern, "", regex=True)

                # Удаление лишних пробелов
                df[column] = df[column].str.replace('\s+', ' ', regex=True).str.strip()
        else:
            print(f"Столбец '{column}' не найден в DataFrame.")

    # Сбрасываем индексы после удаления строк
    df = df.reset_index(drop=True)

    return df


In [92]:
processed_df = preprocess_dataframe(merged_df.copy(), ['title', 'text'])

In [93]:
processed_df

,docid,url,title,modified,lastmodtime,type,domain,status,part,bloc,tags,image_url,pubdate,text,rightcol,snippet
0,1363840,https://lenta.ru/news/2023/01/01/poezd/,в россии поставят точку в споре между пассажир...,1672554120,1673192348,1,1,0,0,48,[159],https://icdn.lenta.ru/images/2023/01/01/09/202...,1672554120,риа новости варвара кошечкина в россии пассажи...,В России поставят точку в споре между пассажир...,Фото: Александр Гальперин / РИА Новости ... на...
1,1363950,https://lenta.ru/news/2023/01/01/perelity/,названы популярные у россиян страны и города д...,1672589820,1673192339,1,1,0,0,48,"[158, 159]",https://icdn.lenta.ru/images/2023/01/01/19/202...,1672589820,anadolu agency via getty images варвара кошечк...,Названы популярные у россиян страны и города д...,Фото: Evgenii Bugubaev / Anadolu Agency ... н...
2,1364096,https://lenta.ru/news/2023/01/02/airastana_trash/,пассажиры самолета попали в зону сильной турбу...,1672658940,1672660748,1,1,0,0,48,[4],https://icdn.lenta.ru/images/2023/01/02/14/202...,1672658940,архивное фото фото александр гальперин риа нов...,Пассажиры самолета попали в зону сильной турбу...,Архивное фото Фото: Александр Гальперин /... и...
3,1364122,https://lenta.ru/news/2023/01/02/turkey_whyyyy...,в турции начал действовать налог на проживание...,1672665180,1673192408,1,1,0,0,48,[158],https://icdn.lenta.ru/images/2023/01/02/16/202...,1672665180,unsplash варвара кошечкина в турции с 1 января...,В Турции начал действовать налог на проживание...,Фото: Artur Voznenko / Unsplash Варвара ... вз...
4,1364168,https://lenta.ru/news/2023/01/02/nalog/,в риге ввели отложенный изза пандемии налог на...,1672680250,1672680250,1,1,0,0,48,[158],https://icdn.lenta.ru/images/2023/01/02/20/202...,1672680250,reuters никита абрамов власти риги решили ввес...,В Риге ввели отложенный из-за пандемии налог н...,Фото: Ints Kalnins / Reuters Никита ... был от...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26395,1754769,https://lenta.ru/news/2024/12/23/remigration/,в ес призвали вернуть всех сирийских беженцев ...,1735011035,1735011035,1,1,0,0,2,[1],https://icdn.lenta.ru/images/2024/12/23/20/202...,1735011035,александр юнгблут фото sebastian willnow dpa g...,В ЕС призвали вернуть всех сирийских беженцев ...,Александр Юнгблут Фото: Sebastian ... режима э...
26396,1754861,https://lenta.ru/news/2024/12/24/v-finlyandii-...,в финляндии закроют используемый россиянами дл...,1735014720,1736370848,1,1,0,0,2,[1],https://icdn.lenta.ru/images/2024/12/24/07/202...,1735014720,globallookpresscom марина совина супермаркет l...,В Финляндии закроют используемый россиянами дл...,Фото: Marijan Murat / Globallookpress.... гран...
26397,1754876,https://lenta.ru/news/2024/12/24/ssha-izuchat-...,сша изучат сценарий полного прекращения помощи...,1735017867,1735020376,1,1,0,1,2,"[1, 48]",https://icdn.lenta.ru/images/2024/12/24/09/202...,1735017867,reuters дарья устьянцева американская разведка...,США изучат сценарий полного прекращения помощи...,Фото: Oleg Petrasiuk / Reuters Дарья ... военн...
26398,1754878,https://lenta.ru/news/2024/12/24/razvedke-ssha...,разведке сша поручили извлечь уроки из конфлик...,1735018160,1735020338,1,1,0,1,2,[1],https://icdn.lenta.ru/images/2024/12/24/09/202...,1735018160,reuters максим габриелян принятый конгрессом и...,Разведке США поручили извлечь уроки из конфлик...,Фото: Oleg Petrasiuk / Reuters Максим ... изуч...


In [102]:
pattern = r'\b(?:строител|ремонт|строительн|строительство|архитектур|подрядчик|здани|проект|инженер|отделк|реконструкц|капитальный|благоустройств)\w*'


mask = (
    processed_df['title'].str.contains(pattern, case=False, na=False, regex=True) |
    processed_df['text'].str.contains(pattern, case=False, na=False, regex=True)
)

processed_df.loc[mask, 'bloc'] = rubrics_w_building.get('building')


In [103]:
processed_df

,docid,url,title,modified,lastmodtime,type,domain,status,part,bloc,tags,image_url,pubdate,text,rightcol,snippet
0,1363840,https://lenta.ru/news/2023/01/01/poezd/,в россии поставят точку в споре между пассажир...,1672554120,1673192348,1,1,0,0,48,[159],https://icdn.lenta.ru/images/2023/01/01/09/202...,1672554120,риа новости варвара кошечкина в россии пассажи...,В России поставят точку в споре между пассажир...,Фото: Александр Гальперин / РИА Новости ... на...
1,1363950,https://lenta.ru/news/2023/01/01/perelity/,названы популярные у россиян страны и города д...,1672589820,1673192339,1,1,0,0,48,"[158, 159]",https://icdn.lenta.ru/images/2023/01/01/19/202...,1672589820,anadolu agency via getty images варвара кошечк...,Названы популярные у россиян страны и города д...,Фото: Evgenii Bugubaev / Anadolu Agency ... н...
2,1364096,https://lenta.ru/news/2023/01/02/airastana_trash/,пассажиры самолета попали в зону сильной турбу...,1672658940,1672660748,1,1,0,0,48,[4],https://icdn.lenta.ru/images/2023/01/02/14/202...,1672658940,архивное фото фото александр гальперин риа нов...,Пассажиры самолета попали в зону сильной турбу...,Архивное фото Фото: Александр Гальперин /... и...
3,1364122,https://lenta.ru/news/2023/01/02/turkey_whyyyy...,в турции начал действовать налог на проживание...,1672665180,1673192408,1,1,0,0,48,[158],https://icdn.lenta.ru/images/2023/01/02/16/202...,1672665180,unsplash варвара кошечкина в турции с 1 января...,В Турции начал действовать налог на проживание...,Фото: Artur Voznenko / Unsplash Варвара ... вз...
4,1364168,https://lenta.ru/news/2023/01/02/nalog/,в риге ввели отложенный изза пандемии налог на...,1672680250,1672680250,1,1,0,0,48,[158],https://icdn.lenta.ru/images/2023/01/02/20/202...,1672680250,reuters никита абрамов власти риги решили ввес...,В Риге ввели отложенный из-за пандемии налог н...,Фото: Ints Kalnins / Reuters Никита ... был от...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26395,1754769,https://lenta.ru/news/2024/12/23/remigration/,в ес призвали вернуть всех сирийских беженцев ...,1735011035,1735011035,1,1,0,0,2,[1],https://icdn.lenta.ru/images/2024/12/23/20/202...,1735011035,александр юнгблут фото sebastian willnow dpa g...,В ЕС призвали вернуть всех сирийских беженцев ...,Александр Юнгблут Фото: Sebastian ... режима э...
26396,1754861,https://lenta.ru/news/2024/12/24/v-finlyandii-...,в финляндии закроют используемый россиянами дл...,1735014720,1736370848,1,1,0,0,2,[1],https://icdn.lenta.ru/images/2024/12/24/07/202...,1735014720,globallookpresscom марина совина супермаркет l...,В Финляндии закроют используемый россиянами дл...,Фото: Marijan Murat / Globallookpress.... гран...
26397,1754876,https://lenta.ru/news/2024/12/24/ssha-izuchat-...,сша изучат сценарий полного прекращения помощи...,1735017867,1735020376,1,1,0,1,2,"[1, 48]",https://icdn.lenta.ru/images/2024/12/24/09/202...,1735017867,reuters дарья устьянцева американская разведка...,США изучат сценарий полного прекращения помощи...,Фото: Oleg Petrasiuk / Reuters Дарья ... военн...
26398,1754878,https://lenta.ru/news/2024/12/24/razvedke-ssha...,разведке сша поручили извлечь уроки из конфлик...,1735018160,1735020338,1,1,0,1,2,[1],https://icdn.lenta.ru/images/2024/12/24/09/202...,1735018160,reuters максим габриелян принятый конгрессом и...,Разведке США поручили извлечь уроки из конфлик...,Фото: Oleg Petrasiuk / Reuters Максим ... изуч...


Найдем соответствие между кодом блока, его названием и кодом в соревновании:

* 1 - 'Общество/Россия' : 0
* 2 - 'Экономика' : 1
* 37 - 'Силовые структуры' : 2
* 3 - 'Бывший СССР' : 3
* 8 - 'Спорт' : 4
* 87 - 'Забота о себе' : 5
* 999 - 'Строительство' : 6
* 48 - 'Туризм/Путешествия' : 7
* 8 - 'Наука и техника' : 8

In [108]:
processed_df = processed_df[processed_df.bloc.isin([1, 2, 37, 3, 8, 87, 999, 48])]

TagsMap = {1 : 0, 2 : 1, 37 : 2, 3 : 3, 8 : 4, 87 : 5, 999 : 6, 48 : 7, 8:8}

processed_df['topic'] = processed_df['bloc'].map(TagsMap)

In [109]:
processed_df.shape

(26400, 17)

In [110]:
processed_df['topic'].value_counts(normalize=True)

topic
8    0.261894
1    0.136742
3    0.133788
2    0.125076
0    0.123598
6    0.074735
5    0.073371
7    0.070795
Name: proportion, dtype: float64

In [112]:
from sklearn.model_selection import train_test_split
test_size = 5000

# Выполняем разделение на тренировочную и тестовую выборки
# Используем параметр stratify для сохранения распределения категорий
train_df, test_df = train_test_split(
    processed_df,
    test_size=test_size,
    random_state=42,  # Для воспроизводимости результатов
    stratify=processed_df['topic']  # Сохраняем пропорции категорий
)

# Проверим размеры полученных выборок
print(f"\nРазмер тренировочной выборки: {len(train_df)} строк")
print(f"Размер тестовой выборки: {len(test_df)} строк")

# Проверим распределение категорий в тренировочной выборке
print("\nРаспределение категорий в тренировочной выборке:")
print(train_df['topic'].value_counts(normalize=True))

# Проверим распределение категорий в тестовой выборке
print("\nРаспределение категорий в тестовой выборке:")
print(test_df['topic'].value_counts(normalize=True))


Размер тренировочной выборки: 21400 строк
Размер тестовой выборки: 5000 строк

Распределение категорий в тренировочной выборке:
topic
8    0.261916
1    0.136729
3    0.133785
2    0.125093
0    0.123598
6    0.074720
5    0.073364
7    0.070794
Name: proportion, dtype: float64

Распределение категорий в тестовой выборке:
topic
8    0.2618
1    0.1368
3    0.1338
2    0.1250
0    0.1236
6    0.0748
5    0.0734
7    0.0708
Name: proportion, dtype: float64


## 2. Машинное обучение

Загружаем данные и обучаем модель на разбиении трейн-тест

In [115]:
processed_df_new = processed_df[~processed_df.text.isna()]

print(len(processed_df), len(processed_df_new))

26400 26400


In [117]:
X = processed_df[['text']]
y = processed_df['topic']

X.shape

(26400, 1)

In [118]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=5000, random_state=42, stratify=processed_df['topic'])

In [119]:
X_train.shape, X_test.shape

((21400, 1), (5000, 1))

In [120]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MaxAbsScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

vec = CountVectorizer() # подбор гиперпараметров очень помогает
vec.fit(X_train['text'])

bow = vec.transform(X_train['text'])  # bow — bag of words (мешок слов)
bow_test = vec.transform(X_test['text'])

print(bow.shape)

scaler = MaxAbsScaler()
bow = scaler.fit_transform(bow)
bow_test = scaler.transform(bow_test)

clf = LogisticRegression(max_iter=200, random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(bow_test)

print(classification_report(y_test, pred))

(21400, 157070)
              precision    recall  f1-score   support

           0       0.82      0.81      0.81       618
           1       0.83      0.90      0.87       684
           2       0.87      0.96      0.91       625
           3       0.84      0.87      0.86       669
           5       0.97      0.98      0.97       367
           6       0.81      0.40      0.54       374
           7       0.90      0.96      0.93       354
           8       0.99      1.00      0.99      1309

    accuracy                           0.89      5000
   macro avg       0.88      0.86      0.86      5000
weighted avg       0.89      0.89      0.89      5000



Загружаем тестовые данные, обучаем итоговую модель и делаем прогноз.

In [121]:
Test = pd.read_csv("test_news.csv")
Test

,content
0,Фото: «Фонтанка.ру»ПоделитьсяЭкс-министру обор...
1,В начале февраля 2023 года в Пушкинском районе...
2,Фото: Andy Bao / Getty Images Анастасия Борисо...
3,"Если вы хотели, но так и не съездили на море л..."
4,Сергей Пиняев Фото: Алексей Филиппов / РИА Нов...
...,...
26270,Фото: РИА Новости Алевтина Запольская Главное ...
26271,Вадим Гутцайт Фото: Sergei CHUZAVKOV / Europea...
26272,Фото: Олег Харсеев / Коммерсантъ Александр Кур...
26273,Владимир Зеленский Фото: Yves Herman / Reuters...


In [130]:
Test_modified = preprocess_dataframe(Test[['content']].copy(), ['content'], primary_column='content', secondary_column='content')

In [131]:
Test_modified

,content,text
0,фото «фонтанкару»поделитьсяэксминистру обороны...,«фонтанкару»фото алина ампелонская «фонтанкару...
1,в начале февраля 2023 года в пушкинском районе...,в начале февраля 2023 года в пушкинском районе...
2,фото andy bao getty images анастасия борисова ...,getty images анастасия борисова международная ...
3,если вы хотели но так и не съездили на море ле...,если вы хотели но так и не съездили на море ле...
4,сергей пиняев фото алексей филиппов риа новост...,сергей пиняев фото алексей филиппов риа новост...
...,...,...
26270,фото риа новости алевтина запольская главное у...,фото риа новости алевтина запольская главное у...
26271,вадим гутцайт фото sergei chuzavkov european u...,вадим гутцайт фото sergei chuzavkov european u...
26272,фото олег харсеев коммерсантъ александр курбат...,коммерсантъ александр курбатов октябрьский рай...
26273,владимир зеленский фото yves herman reuters ва...,владимир зеленский фото yves herman reuters ва...


In [132]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MaxAbsScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

vec = CountVectorizer()
vec.fit(X['text'])

bow = vec.transform(X['text'])  # bow — bag of words (мешок слов)
bow_test = vec.transform(Test_modified['text'])

scaler = MaxAbsScaler()
bow = scaler.fit_transform(bow)
bow_test = scaler.transform(bow_test)

clf = LogisticRegression(max_iter=200, random_state=42)
clf.fit(bow, y)
pred = clf.predict(bow_test)

In [133]:
pred[:10], len(pred)

(array([0, 0, 8, 8, 8, 3, 2, 3, 0, 3]), 26275)

Сохраняем прогноз в файл.

In [134]:
subm = pd.read_csv("base_submission_news.csv")
subm.head()

,topic,index
0,0,0
1,0,1
2,0,2
3,0,3
4,0,4


In [135]:
subm['topic'] = pred

subm.to_csv("bow_logreg_lenta.csv", index=False)

In [136]:
subm['topic'].value_counts(normalize=True)

topic
0    0.287231
8    0.168335
6    0.149876
3    0.117907
2    0.099068
1    0.090542
7    0.058801
5    0.028240
Name: proportion, dtype: float64